In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/fatemasoliman1/desktop/capstone/my_project.json'

In [2]:
#Imports Google Cloud client library
from google.cloud import bigquery

#Instantiates a client for BigQuery service
bigquery_client = bigquery.Client()

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
parameterized_query = """
SELECT
    fullVisitorId,
    visitId,
    visitNumber,
    prod.productListPosition,
    prod.localProductPrice,
    prod.v2ProductCategory,
    prod.v2ProductName
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
    UNNEST(hits) AS hits, 
    UNNEST(hits.product) AS prod 
WHERE
    _TABLE_SUFFIX BETWEEN '%s' AND '%s'
"""
start_date = '20160801'
end_date = '20170801'
# end_date = '20170801'
parameterized_query_df = bigquery_client.query((parameterized_query % (start_date, end_date))).to_dataframe()
parameterized_query_df

KeyboardInterrupt: 

In [ ]:
parameterized_query_df.to_csv("products.csv")

In [6]:
import datetime
parameterized_query_df['date'] = pd.to_datetime(parameterized_query_df['date'])
parameterized_query_df['month'] = parameterized_query_df['date'].dt.month
parameterized_query_df['year'] = parameterized_query_df['date'].dt.year
parameterized_query_df['day'] = parameterized_query_df['date'].dt.weekday

In [7]:
parameterized_query_df=parameterized_query_df.set_index('fullVisitorId')

In [8]:
df_copy = parameterized_query_df.copy()
df = parameterized_query_df.copy()

In [9]:
df.head()

,visitNumber,date,productListPosition,action_type,localProductPrice,v2ProductCategory,v2ProductName,type,hour,minute,timeOnSite,pageviews,medium,source,deviceCategory,browser,country,month,year,day
fullVisitorId,,,,,,,,,,,,,,,,,,,,
4677742971872821654,1,2017-01-20,9,0,1990000,Home/Office/,Google Ballpoint Pen Black,PAGE,8,19,4649.0,110.0,(none),(direct),desktop,Chrome,United States,1,2017,4
4677742971872821654,1,2017-01-20,11,0,28990000,Home/Office/,Metal Texture Roller Pen,PAGE,8,19,4649.0,110.0,(none),(direct),desktop,Chrome,United States,1,2017,4
4677742971872821654,1,2017-01-20,10,0,89990000,Home/Apparel/Men's/Men's-T-Shirts/,Google Men's Performance Polo Grey/Black,PAGE,8,23,4649.0,110.0,(none),(direct),desktop,Chrome,United States,1,2017,4
4677742971872821654,1,2017-01-20,1,0,16990000,Home/Apparel/Men's/Men's-T-Shirts/,Google Men's 100% Cotton Short Sleeve Hero Tee...,PAGE,8,23,4649.0,110.0,(none),(direct),desktop,Chrome,United States,1,2017,4
4677742971872821654,1,2017-01-20,8,0,18990000,Home/Apparel/Men's/Men's-T-Shirts/,Google Men's Vintage Badge Tee Black,PAGE,8,23,4649.0,110.0,(none),(direct),desktop,Chrome,United States,1,2017,4


In [10]:
df['action_type'].unique()

array(['0', '3', '4', '5', '6', '2', '1'], dtype=object)

In [11]:
# pd.set_option('display.max_columns', None)
# df[(df['fullVisitorId']=='3182046558551540109') & ((df['complete_purchase']==1) | (df['add_to_cart']==1) | (df['started_checkout']==1))]

# Label Encoding

In [12]:
from sklearn import preprocessing

In [13]:
le = preprocessing.LabelEncoder()

In [14]:
le.fit(df['type'])
df['type']=le.transform(df['type'])
le.fit(df['medium'])
df['medium']=le.transform(df['medium'])
le.fit(df['deviceCategory'])
df['deviceCategory']=le.transform(df['deviceCategory'])
le.fit(df['browser'])
df['browser'] = le.transform(df['browser'])
le.fit(df['country'])
df['country'] = le.transform(df['country'])
le.fit(df['source'])
df['source'] = le.transform(df['source'])
le.fit(df['v2ProductCategory'])
df['v2ProductCategory'] = le.transform(df['v2ProductCategory'])
le.fit(df['v2ProductName'])
df['v2ProductName'] = le.transform(df['v2ProductName'])

## Dummifying Variables

In [15]:
dum1 = pd.get_dummies(parameterized_query_df['action_type'], prefix='action_type', prefix_sep='__')
dum2 = pd.get_dummies(parameterized_query_df['action_type'], prefix='action_type', prefix_sep='__')
# dum3 = pd.get_dummies(parameterized_query_df['deviceCategory'], prefix='deviceCategory', prefix_sep='__')
# dum4 = pd.get_dummies(parameterized_query_df['browser'], prefix='browser', prefix_sep='__')
# dum5 = pd.get_dummies(parameterized_query_df['country'], prefix='country', prefix_sep='__')
# dum6 = pd.get_dummies(parameterized_query_df['pagePath'], prefix='pagePath', prefix_sep='__')

In [16]:
# print('-'*80)
print('dum1', dum1.columns)
# print('-'*80)
# print('dum2', dum2.columns)
# print('-'*80)
# print('dum3', dum3.columns)
# print('-'*80)
# print('dum4', dum4.columns)
# print('-'*80)
# print('dum5', dum5.columns)
# print('-'*80)
# print('dum6', dum6.columns)

dum1 Index(['action_type__0', 'action_type__1', 'action_type__2', 'action_type__3',
       'action_type__4', 'action_type__5', 'action_type__6'],
      dtype='object')


In [17]:
goal = dum2['action_type__6']

In [18]:
dum1 = dum1.drop('action_type__6', axis = 1)
# dum2 = dum2.drop('medium__(not set)', axis = 1)
# dum3 = dum3.drop('deviceCategory__tablet', axis = 1)
# dum4 = dum4.drop('browser__(not set)', axis = 1)
# dum5 = dum5.drop('country__(not set)', axis = 1)
# dum6 = dum6.drop('type__/asearch.html', axis = 1)

In [19]:
df = pd.concat([df.drop('action_type', axis=1), dum1], axis=1)

In [20]:
# df = pd.concat([df.drop('medium', axis=1), dum2], axis=1)

In [21]:
# df = pd.concat([df.drop('deviceCategory', axis=1), dum3], axis=1)

In [22]:
# df = pd.concat([df.drop('browser', axis=1), dum4], axis=1)

In [23]:
# df = pd.concat([df.drop('country', axis=1), dum5], axis=1)

In [24]:
# df = pd.concat([df.drop('pagePath', axis=1), dum6], axis=1)

In [25]:
#drop action_type because when running decision tree, everything is explained by this variable
# df = df.drop('action_type', axis=1)

In [26]:
#drop complete_purchase because this is the target variable
# df = df.drop('complete_purchase', axis=1)

In [27]:
df = df.drop('date', axis=1)

In [28]:
df = df.drop('action_type__5', axis=1)

In [29]:
df['pageviews']=df['pageviews'].fillna(df['pageviews'].mean())

In [30]:
df['timeOnSite']=df['timeOnSite'].fillna(df['timeOnSite'].mean())

In [31]:
df.isnull().sum()

visitNumber            0
productListPosition    0
localProductPrice      0
v2ProductCategory      0
v2ProductName          0
type                   0
hour                   0
minute                 0
timeOnSite             0
pageviews              0
medium                 0
source                 0
deviceCategory         0
browser                0
country                0
month                  0
year                   0
day                    0
action_type__0         0
action_type__1         0
action_type__2         0
action_type__3         0
action_type__4         0
dtype: int64

In [32]:
df.head()

,visitNumber,productListPosition,localProductPrice,v2ProductCategory,v2ProductName,type,hour,minute,timeOnSite,pageviews,...,browser,country,month,year,day,action_type__0,action_type__1,action_type__2,action_type__3,action_type__4
fullVisitorId,,,,,,,,,,,,,,,,,,,,,
4677742971872821654,1,9,1990000,73,192,1,8,19,4649.0,110.0,...,7,205,1,2017,4,1,0,0,0,0
4677742971872821654,1,11,28990000,73,479,1,8,19,4649.0,110.0,...,7,205,1,2017,4,1,0,0,0,0
4677742971872821654,1,10,89990000,39,277,1,8,23,4649.0,110.0,...,7,205,1,2017,4,1,0,0,0,0
4677742971872821654,1,1,16990000,39,250,1,8,23,4649.0,110.0,...,7,205,1,2017,4,1,0,0,0,0
4677742971872821654,1,8,18990000,39,294,1,8,23,4649.0,110.0,...,7,205,1,2017,4,1,0,0,0,0


In [33]:
df.shape

(18608748, 23)

### Split Data

In [34]:
from sklearn.model_selection import train_test_split
y=goal
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=42)

In [35]:
sum(y_test>0)

22160

# Random Forest

In [36]:
from sklearn import ensemble

In [37]:
randomForest = ensemble.RandomForestClassifier()

In [38]:
randomForest.set_params(n_estimators=50, random_state=42, max_features=10, oob_score=True)
model = randomForest.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("The training error of random forest is: %.5f" %(1 - randomForest.score(X_train, y_train)))
print("The test     error of random forest is: %.5f" %(1 - randomForest.score(X_test, y_test)))

The training error of random forest is: 0.00055
The test     error of random forest is: 0.00205


In [39]:
from sklearn.metrics import accuracy_score, precision_score, f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

Accuracy Score : 0.9979464857481919
Precision Score : 0.759889202060453
F1 Score : 0.73176096214142


In [40]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

Confusion Matrix : 
[[5555524    4941]
 [   6523   15637]]


In [41]:
feature_importance2 = randomForest.feature_importances_
print([t[1] for t in zip(feature_importance2, X_train.columns) if t[0]>0])
feature_importance2

['visitNumber', 'productListPosition', 'localProductPrice', 'v2ProductCategory', 'v2ProductName', 'type', 'hour', 'minute', 'timeOnSite', 'pageviews', 'medium', 'source', 'deviceCategory', 'browser', 'country', 'month', 'year', 'day', 'action_type__0', 'action_type__1', 'action_type__2', 'action_type__3', 'action_type__4']


array([0.03341843, 0.07255382, 0.13468354, 0.0511133 , 0.05930195,
       0.07172799, 0.0474502 , 0.25889714, 0.07138716, 0.06062459,
       0.00765257, 0.00723869, 0.00476019, 0.00567949, 0.0140014 ,
       0.03114663, 0.00461738, 0.0307464 , 0.0126868 , 0.0049115 ,
       0.00062101, 0.01388388, 0.00089595])

In [42]:
imp=zip(X_train.columns, feature_importance2)
res = sorted(imp, key=lambda x: x[1], reverse=True)
res

[('minute', 0.25889714256823365),
 ('localProductPrice', 0.13468354033712354),
 ('productListPosition', 0.07255382023510268),
 ('type', 0.0717279870720904),
 ('timeOnSite', 0.07138716286454411),
 ('pageviews', 0.06062459199293522),
 ('v2ProductName', 0.05930195271372234),
 ('v2ProductCategory', 0.051113300806289196),
 ('hour', 0.04745020190987786),
 ('visitNumber', 0.03341843161012818),
 ('month', 0.031146633160368448),
 ('day', 0.030746395097019),
 ('country', 0.014001398084403687),
 ('action_type__3', 0.013883875285613561),
 ('action_type__0', 0.012686795255486473),
 ('medium', 0.007652569381971782),
 ('source', 0.007238686876510522),
 ('browser', 0.005679491014533311),
 ('action_type__1', 0.004911503557842653),
 ('deviceCategory', 0.004760185873914673),
 ('year', 0.0046173751626054395),
 ('action_type__4', 0.0008959521804826287),
 ('action_type__2', 0.0006210069592007175)]